In [1]:
cd projects/snake_tree/notebooks

/work/FAC/FBM/DBC/cdessim2/default/dmoi/projects/snake_tree/notebooks


In [2]:
%load_ext autoreload
%autoreload 2
import glob
import os
import numpy as np
import sys

sys.path.append( '../src/')
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from compile_treeset import compile_folder , compare_treesets
import plotly.express as px
import plotly.figure_factory as ff
import plotly.express as px
from scipy.stats import wilcoxon, ks_2samp , describe
import treescore

In [3]:
#sample_hogs = glob.glob('../OMA_data_unfiltered/OMA_data/LUCA/*/')
#sample_hogs = sample_hogs[0:10]

In [4]:
#make a test directory and copy the files there
over_write = False

if not os.path.exists('test'):
    os.makedirs('test')
    for hog in sample_hogs:
        os.system('cp -r '+hog+' test/')
    print('copied files')
else:
    if over_write:
        #clear files
        os.system('rm -r test')
        os.makedirs('test')
        for hog in sample_hogs:
            os.system('cp -r '+hog+' test/')
        print('copied files')
    else:
        print('test directory already exists')

test directory already exists


In [5]:
testhogs = glob.glob('test/*/')
print(testhogs)
sample1 = testhogs[9]
samplefiles = glob.glob(sample1+'*')
print(samplefiles)

['test/HOG1077/', 'test/HOG748/', 'test/HOG458/', 'test/HOG411/', 'test/HOG569/', 'test/HOG1118/', 'test/HOG423/', 'test/HOG600/', 'test/HOG475/', 'test/HOG1231/']
['test/HOG1231/3di_xaln_tree.nwk', 'test/HOG1231/fident_1_raw_struct_tree.PP.nwk.rooted', 'test/HOG1231/alntmscore_1_raw_struct_tree.PP.nwk', 'test/HOG1231/alntmscore_0_raw_treescores_struct_tree.json', 'test/HOG1231/alntmscore_0_exp_treescores_struct_tree.json', 'test/HOG1231/alnfident_dist_1_raw_.json', 'test/HOG1231/templateXfoldtree.nx.treefile.rooted', 'test/HOG1231/lddt_distmat.npy', 'test/HOG1231/templateXAA.nx.log', 'test/HOG1231/3di_xaln_tree.PP.nwk.rooted', 'test/HOG1231/logs', 'test/HOG1231/templateXfoldtree.nx.treefile.rooted.final', 'test/HOG1231/3di_xaln_tree.PP.nwk.rooted.final.treescore', 'test/HOG1231/templateX3di.nx.iqtree', 'test/HOG1231/alnfident_dist_3di_AA.json', 'test/HOG1231/lddt_1_exp_struct_tree.PP.nwk', 'test/HOG1231/sequences.aln.muscle.fst.log', 'test/HOG1231/rejected', 'test/HOG1231/fident_0_raw

In [6]:

sample1 = testhogs[9]
treeML = sample1+'sequences.aln.muscle.fst.treefile.rooted'
treeFT = sample1+'fident_1_raw_struct_tree.PP.nwk.rooted'
finalset = sample1 +'finalset.csv'

In [7]:
import toytree
import ete3
from ete3 import PhyloTree
from ete3 import NCBITaxa

ncbi = NCBITaxa()
#ncbi.update_taxonomy_database()


In [8]:

def cut_tree(t ):
    #select one of the children of a duplication node
    recon_dups = t.search_nodes(evoltype="D")
    print(recon_dups)
    for n in t.traverse():
        if n in recon_dups:
            delete = np.random.randint(0,2)
            children = n.children
            if len(children) == 1:
                n = children[0]
                cut_tree(t)
            else:
                if delete == 1:
                    children[0].delete()
                    n = children[0]
                else:
                    children[1].delete()
                    n = children[0]
                cut_tree(t)
    return t

def rf2species(recon_tree, ot_samples = 10):
    for sample in range(ot_samples):    
        ot = recon_tree.copy()

        ot = cut_tree(ot )
        #get the leaf names
        leaves = ot.get_leaf_names()
        lineages = ot.get_leaf_names()
        lineages = [ int(x.split('_')[0]) for x in lineages]
        species_tree = ncbi.get_topology(lineages, intermediate_nodes=False)
        for l in ot.iter_leaves():
            l.name = l.name.split('_')[0]
        RFdist = ot.robinson_foulds(species_tree , unrooted_trees=True)
        rfs.append(RFdist)
    return rfs



In [9]:

import copy

def calc_scores(t , uniprot_csv):
    uniprot_df = pd.read_csv(uniprot_csv)
    tree = toytree.tree(t)
    lineages = treescore.make_lineages(uniprot_df)
    species = treescore.get_species(uniprot_df)
    tree = treescore.label_leaves( tree , lineages , species)
    #tree = treescore.labelwRED(tree.treenode)
    overlap = treescore.getTaxOverlap(tree.treenode)
    taxscore = tree.treenode.score

    #calc descriptive stats on normalized branch lens to see if trees are balanced  
    lengths = np.array([node.dist for node in tree.treenode.traverse()])
    lengths /= np.sum(lengths)
    #calc the root first taxscore

    tree4 = copy.deepcopy(tree)
    treescore.getTaxOverlap_root(tree4.treenode)
    root_score , root_score_nr = treescore.sum_rootscore(tree4.treenode)

    species_set = set()
    #label the leaves with species and number
    for l in tree.treenode.get_leaves():
        if l.sp_num:
            l.name = l.sp_num
            species_set.add(l.sp_num.split('_')[0])

    #get ncbi tree of species set
    species_set = list(species_set)
    species_set = [int(s) for s in species_set]
    
    ncbitree = ncbi.get_topology(species_set, intermediate_nodes=True)
    ncbitree.write(outfile=uniprot_csv.replace('.csv','_ncbi_tree.nwk' ) , format=0)
    #ncbitree = PhyloTree( ncbitree.write() , sp_naming_function=None)
    etetree = PhyloTree( tree.treenode.write( format = 0) , sp_naming_function=None)

    
    #label the leaves with species
    for l in etetree.get_leaves():
        l.species   = l.name.split('_')[0]
    
    
    recon_tree, events = etetree.reconcile(ncbitree)
    recon_dups = recon_tree.search_nodes(evoltype="D")
    recon_losses = recon_tree.search_nodes(evoltype="L")
    recon_speciations = recon_tree.search_nodes(evoltype="S")
    print( 'algo 1' )
    print('dups:', len(recon_dups))
    print('losses:', len(recon_losses))
    print('speciations:', len(recon_speciations))

    #rfs = rf2species(etetree , ot_samples = 10)
    #print('RFs:', rfs)


    print( 'algo 2' )
    events = etetree.get_descendant_evol_events()
    dups = etetree.search_nodes(evoltype="D")
    losses = etetree.search_nodes(evoltype="L")
    speciations = etetree.search_nodes(evoltype="S")    
    print('dups:', len(dups))
    print('losses:', len(losses))
    print('speciations:', len(speciations))

    scores = {}
    #measure the distances of leaves to root
    distances = np.array([ node.get_distance(tree.treenode) for node in tree.treenode.get_leaves() ])
    distances_norm = distances / np.mean(distances)
    scores[t] = {'score': taxscore, 'stats': describe(lengths) , 'ultrametricity':  describe(distances), 
                    'ultrametricity_norm':  describe(distances_norm) , 'root_score': root_score , 'root_score_nr': root_score_nr  , 
                    'RECON_speciations': len(recon_speciations) , 'RECON_dups': len(recon_dups) , 'RECON_losses': len(recon_losses) ,
                    'SO_speciations':len(speciations) ,'SO_dups': len(dups) , 'SO_losses': len(losses)  }
    return scores

In [10]:
for sample in testhogs:
    print( sample)
    treeML = sample+'sequences.aln.muscle.fst.treefile.rooted'
    treeFT = sample+'fident_1_exp_struct_tree.PP.nwk.rooted'
    finalset = sample +'finalset.csv'

    print('ML')
    scores = calc_scores(treeML , finalset)
    print(scores)
    print('FT')
    scores = calc_scores(treeFT , finalset)
    print(scores)

test/HOG1077/
ML
root scoring
root score 4623
root score_nr 3909
algo 1
dups: 23
losses: 3389
speciations: 35
algo 2
dups: 12
losses: 0
speciations: 46
{'test/HOG1077/sequences.aln.muscle.fst.treefile.rooted': {'score': 626, 'stats': DescribeResult(nobs=117, minmax=(2.991111384783379e-08, 0.0726053954572658), mean=0.00854700854700855, variance=0.00017982310920735504, skewness=2.671951856873734, kurtosis=7.301067678776121), 'ultrametricity': DescribeResult(nobs=59, minmax=(1.707441299, 4.3434943268), mean=2.139202316576271, variance=0.16713058024887115, skewness=3.168209235115583, kurtosis=13.302032524293164), 'ultrametricity_norm': DescribeResult(nobs=59, minmax=(0.7981672821543633, 2.030427086369106), mean=1.0, variance=0.03652180224239135, skewness=3.1682092351155826, kurtosis=13.302032524293164), 'root_score': 4623, 'root_score_nr': 3909, 'RECON_speciations': 35, 'RECON_dups': 23, 'RECON_losses': 3389, 'SO_speciations': 46, 'SO_dups': 12, 'SO_losses': 0}}
FT
root scoring
root score 

In [27]:
import subprocess
#running astral
def run_astral( phylo , st,  mapping, output ,root  ,outfmt = '2', astralpath = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/software/ASTER-Linux/bin/astral-pro3' ):
    logfile = output+'.log'
    cmd = astralpath + ' -c {st} -a {mapping} -u {outfmt} -i {phylo} -o {output} -C -T -E --root {root} 2>{logfile}'.format(phylo = phylo , st=st, mapping = mapping , output = output , root=root, logfile = logfile , outfmt = outfmt)
    print(cmd)
    subprocess.run(cmd , shell = True)
    return output , logfile

def prepare_astral_input(uniprot_df , output):
    finalset = pd.read_csv(uniprot_df)
    finalset['species'] = finalset['Taxonomic lineage (Ids)'].map(lambda x: x.split(',')[-1].split('(')[0].strip())
    finalset['species'] = finalset['species'].map(lambda x: x.split('_')[0])
    mapper = dict(zip(finalset['Entry'], finalset['species']))
    with open(uniprot_df.replace('.csv','_speciesmap.txt' ) , 'w') as f:
        for k,v in mapper.items():
            f.write('{}\t{}\n'.format(k,int(v)))
    
    #get ncbi tree of species set
    species_set = list(finalset.species.unique())
    species_set = [int(s) for s in species_set]
    st = ncbi.get_topology(species_set, intermediate_nodes=False)
    st.name = 'root'
    #write with internal node names

    st.write(outfile=uniprot_df.replace('.csv','_ncbi_tree.nwk' ) , format= 1 )
    return mapper , uniprot_df.replace('.csv','_speciesmap.txt' )  , uniprot_df.replace('.csv','_ncbi_tree.nwk' ) 

In [28]:
from Bio import Phylo
import re
from scipy.stats import describe

"""

q1,q2,q3: these three values show quartet support (as defined in the description of -t 1) for the main topology, the first alternative, and the second alternative, respectively.
f1, f2, f3: these three values show the total number of quartet trees in all the gene trees that support the main topology, the first alternative, and the second alternative, respectively.
pp1, pp2, pp3: these three show the local posterior probabilities (as defined in the description of -t 4) for the main topology, the first alternative, and the second alternative, respectively.

"""

def print_tree(tree):
    tree = ete3.Tree(tree)
    print(tree)

def parse_astral(string , index = 0):
    #'[pp1=0.284784;pp2=0.426968;pp3=0.288248;f1=0.000000;f2=0.263158;f3=0.008772;q1=0.000000;q2=0.967742;q3=0.032258]'
    string = string.replace('[','').replace(']','')
    string = string.split(';')
    string = {s.split('=')[0]:float(s.split('=')[1]) for s in string}
    #to dataframe
    string = pd.DataFrame(string , index = [index])
    return string

def retastral_support( astral_file):
    with open(astral_file , 'r') as f:
        nstring = f.read()
    #find all quoted strings with single quotes
    quoted = re.findall(r"'(.*?)'", nstring)
    dfs = [ parse_astral(q,i) for i,q in enumerate(quoted)]
    dfs = pd.concat(dfs)
    #make sure pp1 does not contain nan
    return dfs

def verify_map(mapping, tree):
    t = ete3.Tree(tree)
    for l in t.get_leaves():
        if l.name not in mapping.keys():
            print('missing', l.name)
            return False
    return True 

def return_astral_score(astrolout, logfile):
    #get dup loss and speciation events from astral logfile
    with open(logfile , 'r') as f:
        for l in f:
            if '#Duploss' in l:
                duploss = int(l.split(' ')[-1])
    astraldf = retastral_support(astrolout)
    #count nans 
    nancount = astraldf['pp1'].isna().sum()
    #get summary stats
    astral_stas = describe(astraldf['pp1'].dropna())
    return { 'mean': astral_stas.mean , 'std': astral_stas.variance ,'skewness':astral_stas.skewness, 'nancount': nancount , 'duploss': duploss }

for sample in testhogs:
    speciesmap, speciesmap_file , ncbitree = prepare_astral_input(sample+'finalset.csv' , sample+'speciesmap.txt')
    FT = sample+'fident_1_exp_struct_tree.PP.nwk.rooted'
    ML = sample+'sequences.aln.muscle.fst.treefile.rooted'
    t = PhyloTree(ncbitree , sp_naming_function=None)
    #find root node name
    root = t.get_tree_root()
    root.name = 'root'
    root.write(outfile=ncbitree , quoted_node_names = False, format=1)
    #open ncbi tree
    #t = PhyloTree(ncbitree )
    #find root node name
    #t.write(outfile=ncbitree , quoted_node_names = False, format=1)
    print('FT')
    output , log = run_astral(  FT , ncbitree, speciesmap_file , sample+'FT_astral_support.tre' , 'root')
    print( return_astral_score(output, log))
    print('ML')
    output , log = run_astral( ML , ncbitree,speciesmap_file , sample+'ML_astral_support.tre', 'root')
    print( return_astral_score(output, log))

FT
/work/FAC/FBM/DBC/cdessim2/default/dmoi/software/ASTER-Linux/bin/astral-pro3 -c test/HOG1077/finalset_ncbi_tree.nwk -a test/HOG1077/finalset_speciesmap.txt -u 2 -i test/HOG1077/fident_1_exp_struct_tree.PP.nwk.rooted -o test/HOG1077/FT_astral_support.tre -C -T -E --root root 2>test/HOG1077/FT_astral_support.tre.log
{'mean': 0.32970871428571424, 'std': 0.0005210136075714289, 'skewness': -1.015143816812902, 'nancount': 0, 'duploss': 14}
ML
/work/FAC/FBM/DBC/cdessim2/default/dmoi/software/ASTER-Linux/bin/astral-pro3 -c test/HOG1077/finalset_ncbi_tree.nwk -a test/HOG1077/finalset_speciesmap.txt -u 2 -i test/HOG1077/sequences.aln.muscle.fst.treefile.rooted -o test/HOG1077/ML_astral_support.tre -C -T -E --root root 2>test/HOG1077/ML_astral_support.tre.log
{'mean': 0.32563671428571433, 'std': 0.003455933421904763, 'skewness': -1.5878599953867245, 'nancount': 0, 'duploss': 15}
FT
/work/FAC/FBM/DBC/cdessim2/default/dmoi/software/ASTER-Linux/bin/astral-pro3 -c test/HOG748/finalset_ncbi_tree.nw

: 